<a href="https://colab.research.google.com/github/adeepH/DravidianOffensive/blob/main/Notebooks/MultichannelBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install googletrans==3.1.0a0

In [2]:
from transformers import BertModel
import os
from googletrans import Translator
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [ ]:
!git clone https://github.com/adeepH/DravidianOffensive.git


In [4]:
train = pd.read_csv('/content/DravidianOffensive/Datasets/kannada_offensive_train (1).csv', names=['tweets','label'],sep='\t')
train['labels']=LabelEncoder().fit_transform(train['label'])  
train=train.drop(columns='label')
train.head(10)

,tweets,labels
0,Tik tok alli jagala madtidralla adra baggenu o...,0
1,Anyone from kerala here,5
2,Movie rerelease madi plss,0
3,Amazon prime alli bittidira....yella manele no...,0
4,Guru sure news nanu tik tok dawn lod madeda ya...,0
5,ಸುದೀಪ್ ಸರ್ ಅಂಡ್ ದರ್ಶನ್ ಸರ್ ಅವರಿಗೆ ಇರೋ ಫ್ಯಾನ್ಸ್...,0
6,Ade old same story,1
7,Superb rakshit sir sobg superb macking,5
8,Hai Neel (prithvi) wonderful movie dia,5
9,ಏನ್ ಗುರು ನಿನ್ನ ಚರಿತ್ರೆ ಸೃಷ್ಟಿಸುವ ಅವತಾರ ತುಂಬಾ ಚ...,0


In [14]:
train['tweets'][5]

'ಸುದೀಪ್ ಸರ್ ಅಂಡ್ ದರ್ಶನ್ ಸರ್ ಅವರಿಗೆ ಇರೋ ಫ್ಯಾನ್ಸ್ ಫಾಲೋಯಿಂಗ್ ರಕ್ಷಿತ್ ಶೆಟ್ಟಿ ಇಲ್ಲ ಅಂದ್ರು ಸಾಂಗ್ 2m  ಮತ್ತು ಟೀಸರ್ 8.5.m ಬೆಂಕಿ ಗುರು ಮೂವಿ  ವೇಟಿಂಗ್'

In [17]:
translator = Translator()
k = translator.translate(train['tweets'][5])

In [18]:
k.text

'Sudeep sir and Darshan sir have no ero fans following Rakshit Shetty Andrew Song 2m and teaser 8.5.m fire guru movie waiting'

In [ ]:
translator = Translator() 
kan_to_en = []
for i in range(0, len(train)):
    trans = translator.translate(train['tweets'][i], dest='en')
    kan_to_en.append(trans.text)
    if i % 500 == 0:
        print(i)

In [ ]:
kan_to_en

In [ ]:
len(kan_to_en)

In [ ]:
train['translation'] = kan_to_en

In [27]:
import pandas as pd
from torch.utils.data import Dataset,DataLoader

class RFDataset(Dataset):
  def __init__(self,text, translation, label,tokenizer1, tokenizer2, max_len):
    self.text = text
    self.translation = translation
    self.label = label
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self,item):
    text = str(self.text[item])
    translation = str(self.translation[item])
    label = self.label[item]

    encoding1 = self.tokenizer1.encode_plus(
        text,
        add_special_tokens=True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask= True,
        return_tensors='pt',
        truncation=True
    )
    
    encoding2 = self.tokenizer2.encode_plus(
        translation,
        add_special_tokens=True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask= True,
        return_tensors='pt',
        truncation=True
    )
    return {
        'text' : text,
        'translation': translation,
        'input_ids1' : encoding1['input_ids'].flatten(),
        'input_ids2' : encoding2['input_ids'].flatten(),
        'attention_mask1' : encoding['attention_mask'].flatten(),
        'attention_mask2' : enocding['attention_mask'].flatten(),
        'label' : torch.tensor(label,dtype=torch.long)

    }

In [28]:
def create_data_loader(df, tokenizer1, tokenizer2, max_len, batch_size):
    
    ds =  RFDataset(
        text = df.tweets.to_numpy(),
        translation = df.transation.to_numpy(),
        label = df.labels.to_numpy(),
        tokenizer1 = tokenizer1,
        tokenizer2 = tokenizer2,
    )
    
    return DataLoader(ds,
                      batch_size = batch_size,
                      shuffle=True,
                      num_workers = 2)

In [30]:
import torch.nn as nn
class LinearWeightedAvg(nn.Module):
    def __init__(self, n_inputs):
        super(LinearWeightedAvg, self).__init__()
        self.weights = nn.ParameterList([nn.Parameter(torch.randn(1)) for i in range(n_inputs)])

    def forward(self, input):
        res = 0
        for emb_idx, emb in enumerate(input):
            res += emb * self.weights[emb_idx]
        return res


In [37]:
import torch
import torch.nn as nn
from transformers import AutoModel
class MultichannelBERT(nn.Module):

  def __init__(self, n_classes):
    super(MultichannelBERT, self).__init__()
    self.bert = AutoModel.from_pretrained('bert-base-uncased')
    self.mbert = AutoModel.from_pretrained('bert-base-multilingual-cased')
    self.fc1 = nn.Linear(self.bert.config.hidden_size, 128)
    self.fc2 = nn.Linear(self.mbert.config.hidden_size,128)
    self.relu = nn.ReLU()
    self.fc3 = nn.Linear(256,n_classes)
    self.drop = nn.Dropout(p=0.2)

  def forward(self,input_ids1, attention_mask1, input_ids2, attention_mask2):

    _, bert_output = self.bert(
      input_ids = input_ids1,
      attention_mask = attention_mask1
    )

    _, mbert_output = self.mbert(
        input_ids = input_ids2,
        attention_mask = attention_mask2
    )
    
    bert_out = self.fc1(bert_output)
    mbert_out = self.fc2(mbert_output)

    #Merges both the outputs 
    merged = torch.cat((bert_out, mbert_out),1)
    act = self.relu(merged)
    out = self.drop(act)
    return self.fc3(out)

In [38]:
MultichannelBERT(5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MultichannelBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [41]:
model = MultichannelBERT(5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predic

In [42]:
print(model)

MultichannelBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T